# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

import pkg_resources
import azureml.core

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

import joblib

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model

#To check core SDK version number
print("SDK Version:",azureml.core.VERSION)

SDK Version: 1.20.0


# Workspace and Azure ML Experiment Creation

In [8]:
ws = Workspace.from_config()
exp=Experiment(workspace=ws,name="heart-failure")
project_folder="./my-capstone-project"
print("Workspace name :"+ ws.name,
      "Resource group:"+ ws.resource_group, 
      "Azure Region:"+ ws.location, 
      "Subscription Id:"+ ws.subscription_id, sep= '\n')

run=exp.start_logging()

Workspace name :quick-starts-ws-138664
Resource group:aml-quickstarts-138664
Azure Region:southcentralus
Subscription Id:6971f5ac-8af1-446e-8034-05acea24681f


# Creation or Attaching of Cluster

In [11]:
#choosing a name for compute cluster
compute_cluster_name="demo-1-cluster"
#verifying if the cluster exists already
try:
    Compute_target=ComputeTarget(workspace=ws,name=compute_cluster_name)
    print("Found existing cluster ! Use it ! No need of creating another.")
except ComputeTargetException:
    print("Creating a new compute cluster... ")
    config_of_compute=AmlCompute.provisioning_configuration(vm_size="Standard_DS3_v2",max_nodes=4)
    Compute_target=ComputeTarget.create(ws,compute_cluster_name,config_of_compute)
    Compute_target.wait_for_completion(show_output=True)
#retrieving compute cluster details
print(Compute_target.get_status().serialize())

Found existing cluster ! Use it ! No need of creating another.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-12T07:58:01.818000+00:00', 'errors': None, 'creationTime': '2021-02-12T07:57:57.729645+00:00', 'modifiedTime': '2021-02-12T07:58:13.680165+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [13]:
datastore=ws.get_default_datastore()

In [14]:
datastore.upload(src_dir='./',target_path='./')

Uploading an estimated of 9 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 9
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 9
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 3 files out of an estimated total of 9
Uploading ./automl.ipynb.amltmp
Uploaded ./automl.ipynb.amltmp, 4 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 5 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb.amltmp
Uploaded ./hyperparameter_tuning.ipynb.amltmp, 6 files out of an estimated total of 9
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2021-1-12-12-18-8.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2021-1-12-12-18-8.ipynb, 7 files out of an estimated total of 9
Uploading ./.ipynb_aml_checkpoints/hyperparameter_tuning-checkpoint2021-1-12-12-18-32.ipynb
Uploaded ./.ipynb_aml_checkpoints/hyperparameter_tuning

$AZUREML_DATAREFERENCE_workspaceblobstore

In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory
dataset=pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found=False
key="heart-failure-prediction"
iff  key  in
datapath_automl="https://github.com/Ankita03-dell/AZMLND_Capstone_Trial1/blob/main/heart_failure_clinical_records_dataset.csv"
ds_automl=TabularDatasetFactory.from_delimited_files(path=datapath_automl)

DatasetValidationError: DatasetValidationError:
	Message: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
The Dataflow produced no records.| session_id=926315cc-0c77-4c7b-b7de-7633724bd6d2
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot load any data from the specified path. Make sure the path is accessible and contains data.\nThe Dataflow produced no records.| session_id=926315cc-0c77-4c7b-b7de-7633724bd6d2"
    }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig()

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service